<a href="https://colab.research.google.com/github/RonildoSilva/datasets/blob/main/big_black_forecast_da_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime

! pip install flood-forecast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 117 kB 8.1 MB/s 
     |████████████████████████████████| 5.9 MB 36.6 MB/s 
     |████████████████████████████████| 1.8 MB 45.7 MB/s 
     |████████████████████████████████| 2.8 MB 37.8 MB/s 
     |████████████████████████████████| 28.8 MB 1.3 MB/s 
     |████████████████████████████████| 564 kB 74.1 MB/s 
     |████████████████████████████████| 201 kB 62.7 MB/s 
     |████████████████████████████████| 1.1 MB 37.2 MB/s 
     |████████████████████████████████| 181 kB 49.2 MB/s 
     |████████████████████████████████| 158 kB 19.3 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 3.5 MB 43.0 MB/s 
     |████████████████████████████████| 157 kB 53.6 MB/s 
     |████████████████████████████████| 157 kB 32.4 MB/s 
     |████████████████████████████████| 157 kB 40.8 MB/s 
     |████████████████████████████

In [2]:
! pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=f90b0b99df90f4282683dee76cdf77c18201cead658f77aed5b27a7e7487852a
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [3]:
import wget
import os.path

In [4]:
#! unzip 01010500FVE_flow.csv.zip

In [5]:
import wget
import os.path

if(not os.path.exists('df_train_keras_tk.csv')):
    wget.download('https://raw.githubusercontent.com/RonildoSilva/datasets/main/df_train_keras_tk.csv')

if(not os.path.exists('df_val_keras_tk.csv')):
    wget.download('https://raw.githubusercontent.com/RonildoSilva/datasets/main/df_val_keras_tk.csv')

if(not os.path.exists('df_test_keras_tk.csv')):
    wget.download('https://raw.githubusercontent.com/RonildoSilva/datasets/main/df_test_keras_tk.csv')

In [6]:
"""
from google.colab import files
uploaded = files.upload()
"""

'\nfrom google.colab import files\nuploaded = files.upload()\n'

In [7]:
! ls

df_test_keras_tk.csv  df_train_keras_tk.csv  df_val_keras_tk.csv  sample_data


Merge

In [8]:
import pandas as pd 
df = pd.concat([
    pd.read_csv('df_train_keras_tk.csv'),
    pd.read_csv('df_val_keras_tk.csv'),
    pd.read_csv('df_test_keras_tk.csv')
])

In [9]:
print(
    pd.read_csv('df_train_keras_tk.csv').shape,
    pd.read_csv('df_val_keras_tk.csv').shape,
    pd.read_csv('df_test_keras_tk.csv').shape
)

(12363, 20) (3825, 20) (4216, 20)


In [10]:
pd.to_datetime(pd.read_csv('df_val_keras_tk.csv')['datetime']).dt.strftime('%Y-%m-%d').max()

'2012-09-20'

In [11]:
TRAIN_START = 0
TRAIN_END = pd.read_csv('df_train_keras_tk.csv').shape[0]
#TRAIN_END = 100

VAL_START = TRAIN_END + 1
VAL_END = VAL_START + pd.read_csv('df_val_keras_tk.csv').shape[0]
#VAL_END = VAL_START + 50

TEST_START = VAL_END + 1
TEST_END = TEST_START + pd.read_csv('df_test_keras_tk.csv').shape[0] - 1
#TEST_END = TEST_START + 50

In [12]:
print(TRAIN_START, TRAIN_END, VAL_START, VAL_END, TEST_START, TEST_END)

0 12363 12364 16189 16190 20405


In [13]:
DATETIME_START = pd.to_datetime(pd.read_csv('df_test_keras_tk.csv')['datetime']).dt.strftime('%Y-%m-%d').min()
DATETIME_END = pd.to_datetime(pd.read_csv('df_test_keras_tk.csv')['datetime']).dt.strftime('%Y-%m-%d').max()

#DATETIME_START = pd.to_datetime(pd.read_csv('df_test_keras_tk.csv')['datetime']).dt.strftime('%Y-%m-%d')[TEST_START]
#DATETIME_END = pd.to_datetime(pd.read_csv('df_test_keras_tk.csv')['datetime']).dt.strftime('%Y-%m-%d %H:%m:%S+00:00')[TEST_END]

DATETIME_START, DATETIME_END

('2012-07-30', '2014-01-03')

In [14]:
df['datetime'] = pd.to_datetime(df['datetime']).dt.strftime('%Y-%m-%d')
#df['datetime'] = pd.to_datetime(df['datetime']).dt.strftime('%Y-%m-%d %H:%m:%S+00:00')
#2000-04-04 04:00:00+00:00

In [15]:
#df = df[["TimeConclusion", "Duration", "datetime"]]
df.to_csv('df_full.csv', index=False)

In [16]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,datetime,Duration,PassedTime,Step,TimeConclusion
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010-01-13,0.0000,0.0000,1,31.0087
1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2010-01-29,16.0084,15.0002,2,15.0003
2,1,2,3,0,0,0,0,0,0,0,0,0,0,0,0,2010-01-29,0.0001,15.0003,3,15.0002
3,1,2,3,5,0,0,0,0,0,0,0,0,0,0,0,2010-02-13,15.0002,31.0087,4,0.0000
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010-01-13,0.0000,0.0000,1,30.9822


In [17]:
"""
!pip install -Uqq ipdb
import ipdb
%pdb on
"""

'\n!pip install -Uqq ipdb\nimport ipdb\n%pdb on\n'

In [18]:
FEATURE_COLS = df.columns.tolist()
FEATURE_COLS.remove('datetime')

TARGET_COL = ["TimeConclusion"]

In [19]:
DIMENSION_FEAT = len(FEATURE_COLS)

In [20]:
DIMENSION_FEAT

19

In [21]:
print(FEATURE_COLS, sep=' ')

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', 'Duration', 'PassedTime', 'Step', 'TimeConclusion']


In [22]:
print(TARGET_COL)

['TimeConclusion']


Config

In [23]:
PROJECT_NAME = 'hd17-flow-forecast'

In [28]:
def make_config_file(flow_file_path, gage_id, station_id, weight_path=None, pretrained=[]):
    run = wandb.init(project=PROJECT_NAME)
    wandb_config = run.config
    the_wandb_c = run.config
    print(wandb_config)
    the_config4 = {"model_name": "DARNN",
        "model_type": "PyTorch",
        "model_params": {
        "n_time_series": (2+DIMENSION_FEAT),
        "forecast_history": 15,
        "hidden_size_encoder":18,
        "decoder_hidden_size": 18,
        "out_feats": 1,
        "dropout": 0.5,
        "gru_lstm": False},
       "dataset_params":{"class": "default",
       "num_workers":2,
       "pin_memory": True,
       "training_path": flow_file_path,
       "validation_path": flow_file_path,
       "test_path": flow_file_path,
       "batch_size": 500,
       "forecast_history": (15 - 1),
       "forecast_length": 1,
       "scaler": "RobustScaler",
       "train_start": TRAIN_START,
       "train_end": TRAIN_END,
       "valid_start": VAL_START,
       "valid_end": VAL_END,
       "sort_column": "datetime",# "hour_updated",
       "test_start": TEST_START,
       "test_end": TEST_END,
       "target_col": TARGET_COL,
       "relevant_cols": FEATURE_COLS, 
       "interpolate":{
           "method":"back_forward_generic",
           "params":{
               "relevant_columns":FEATURE_COLS}
           },
      "feature_param":
          {
            "datetime_params":{
            "day":"numerical",
            "month": "numerical"
            }
          }
       },
    "training_params":
    {
       "criterion":"L1", #"MSE"
       "optimizer": "SGD", # DONE
       #"criterion_params":{"baseline_method":"mean"},
    "optim_params":{
       "lr": 0.01, #[0.0001,] (0.001)
    },
       "epochs": 10,
       "batch_size": 1000
    },
    "early_stopping": {
        "patience": 5
    },
    "GCS": False,
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE", "L1"],
   "inference_params":
   {     
          "datetime_start":DATETIME_START,
          "hours_to_forecast": 600, 
          "num_prediction_samples": 100,
          "test_csv_path":flow_file_path,
          "decoder_params":{
            "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1},
          "dataset_params":{
             "file_path": flow_file_path,
             "sort_column": "datetime",
             "scaling": "RobustScaler",
             "forecast_history": (15 - 1),
             "forecast_length":1,
             "relevant_cols": FEATURE_COLS,
             "target_col": TARGET_COL,
             "interpolate_param":{
                 "method":"back_forward_generic",
                 "params":{"relevant_columns":FEATURE_COLS}
                 }, 
            "feature_params":
        {
         "datetime_params":{
            "day":"numerical",
            "month": "numerical"         
     }
             }
          }
          } 
    }

      
    if weight_path:
        the_config4["weight_path"] = weight_path
    wandb.config.update(the_config4)
    print("config made")
    return the_config4
  
wandb_sweep_config_full = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "forecast_length":{
            "values":[1]}
            ,
        "batch_size": {
            "values": [500]
        },
        "lr":{
            "values":[0.01]
        },
        "forecast_history":{
            "values":[15]
        },
        "out_seq_length":{
            "values":[1]
        },
        "hidden_encoder":
        {
            "values":[18]
        },
        "hidden_decoder":
        {
            "values":[18]
        }, 
        "dropout":
        {
            "values": [0.1]
        }
  
    }
}

In [29]:
import wandb
from flood_forecast.trainer import train_function


sweep_id = wandb.sweep(wandb_sweep_config_full, project=PROJECT_NAME)

file_path = "df_full.csv"

#wandb.agent(sweep_id, lambda: train_function("PyTorch", make_config_file(file_path, "01010500", "FVE", None)))

model = train_function("PyTorch", make_config_file(file_path, "01010500", "FVE", None))

Create sweep with ID: r8lva7vl
Sweep URL: https://wandb.ai/ronildosilva/hd17-flow-forecast/sweeps/r8lva7vl


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▃▂▂▁▁▁▁▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▁▂▂▂
epoch,51
loss,0.25322


{}
config made
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['day', 'month']
scaling now
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['day', 'month']
scaling now
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['day', 'month']
scaling now
Using Wandb config:
{'model_name': 'DARNN', 'model_type': 'PyTorch', 'model_params': {'n_time_series': 21, 'forecast_history': 15, 'hidden_size_encoder': 18, 'decoder_hidden_size': 18, 'out_feats': 1, 'dropout': 0.5, 'gru_lstm': False}, 'dataset_params': {'class': 'default', 'num_workers': 2, 'pin_memory': True, 'training_path': 'df_full.csv', 'validation_path': 'df_full.csv', 'test_path': 'df_full.csv', 'batch_size': 500, 'forecast_history': 14, 'forecast_length': 1, 'scaler': 'RobustScaler', 'train_start': 0, 'train_end': 12363, 'valid_start': 12364, 'valid_end': 16189, 'sor

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
Using a non-full backward hook when outputs are generated by different autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_output. Please use register_full_backward_hook to get the documented behavior.
Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1405.)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1053: UserWarning:

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input

plotting with CI now
Now plotting final plots


In [30]:
#model.test_data.df

**grateful-glitter-78**

- 2.980451452524741

**clean-glade-120**

- 2.877648783491315